# Chapter 1: Hello Transformers

Requirements:

  - ipywidgets
  - transformers
  - ipykernel
  - matplotlib
  - scikit-learn
  - pytorch::pytorch
  - torchaudio
  - torchvision
  - sentencepiece
  - umap-learn

If you are on the `mydocker.centralesupelec.fr` server, you can install the requirements by running the following command:

In [ ]:
# !pip install datasets umap-learn

In [2]:
import pprint

import pandas as pd
from transformers import pipeline, set_seed

import utils

In [3]:
pp = pprint.PrettyPrinter(indent=4, width=90, compact=True)

In [2]:
utils.setup_chapter()

No GPU was detected! This notebook can be *very* slow without a GPU 🐢
Using transformers v4.37.1
Using datasets v2.16.1


<img alt="transformer-timeline" caption="The transformers timeline" src="images/chapter01_timeline.png" id="transformer-timeline"/>

## The Encoder-Decoder Framework

<img alt="rnn" caption="Unrolling an RNN in time." src="images/chapter01_rnn.png" id="rnn"/>

<img alt="enc-dec" caption="Encoder-decoder architecture with a pair of RNNs. In general, there are many more recurrent layers than those shown." src="images/chapter01_enc-dec.png" id="enc-dec"/>

## Attention Mechanisms

<img alt="enc-dec-attn" caption="Encoder-decoder architecture with an attention mechanism for a pair of RNNs." src="images/chapter01_enc-dec-attn.png" id="enc-dec-attn"/> 

<img alt="attention-alignment" width="500" caption="RNN encoder-decoder alignment of words in English and the generated translation in French (courtesy of Dzmitry Bahdanau)." src="images/chapter02_attention-alignment.png" id="attention-alignment"/> 

<img alt="transformer-self-attn" caption="Encoder-decoder architecture of the original Transformer." src="images/chapter01_self-attention.png" id="transformer-self-attn"/> 

## Transfer Learning in NLP

<img alt="transfer-learning" caption="Comparison of traditional supervised learning (left) and transfer learning (right)." src="images/chapter01_transfer-learning.png" id="transfer-learning"/>  

<img alt="ulmfit" width="500" caption="The ULMFiT process (courtesy of Jeremy Howard)." src="images/chapter01_ulmfit.png" id="ulmfit"/>

## Hugging Face Transformers: Bridging the Gap

## A Tour of Transformer Applications

In [3]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### Text Classification

In [4]:
classifier = pipeline("text-classification")

In [5]:
outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901547


### Named Entity Recognition

In [6]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556570,Mega,208,212
4,PER,0.590256,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


### Question Answering 

In [7]:
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


### Summarization

In [8]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
pp.pprint(outputs[0]["summary_text"])

/Users/evrard/miniconda3/envs/nlp-tr/lib/python3.12/site-packages/transformers/generation/utils.py:1158: UserWarning: Unfeasible length constraints: `min_length` (56) is larger than the maximum possible length (45). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


(' Bumblebee ordered an Optimus Prime action figure from your online store in '
 'Germany. Unfortunately, when I opened the package, I discovered to my horror '
 'that I had been sent an action figure of Megatron instead.')


### Translation

In [9]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
pp.pprint(outputs[0]["translation_text"])

/Users/evrard/miniconda3/envs/nlp-tr/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


('Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur '
 'aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket '
 'öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action '
 'Figur von Megatron geschickt worden war! Als lebenslanger Feind der '
 'Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu '
 'lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur '
 'habe ich bestellt. Eingeschlossen sind Kopien meiner Aufzeichnungen über '
 'diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee.')


### Text Generation

In [10]:
set_seed(42) # Set the seed to get reproducible results

In [11]:
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
pp.pprint(outputs[0]['generated_text'])

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('Dear Amazon, last week I ordered an Optimus Prime action figure from your '
 'online store in Germany. Unfortunately, when I opened the package, I '
 'discovered to my horror that I had been sent an action figure of Megatron '
 'instead! As a lifelong enemy of the Decepticons, I hope you can understand '
 'my dilemma. To resolve the issue, I demand an exchange of Megatron for the '
 'Optimus Prime figure I ordered. Enclosed are copies of my records concerning '
 'this purchase. I expect to hear from you soon. Sincerely, Bumblebee.\n'
 '\n'
 'Customer service response:\n'
 'Dear Bumblebee, I am sorry to hear that your order was mixed up. Please '
 'inform me immediately that it was not completed. I will try to locate the '
 'items you ordered in your shop on the day. As soon as I have received that '
 'information (this was only on Wednesday which means that it could take too '
 'long to process the results of the exchange), I will refund the order and '
 'refund my entire purchase pr

## The Hugging Face Ecosystem

<img alt="ecosystem" width="500" caption="An overview of the Hugging Face ecosystem of libraries and the Hub." src="images/chapter01_hf-ecosystem.png" id="ecosystem"/>

### The Hugging Face Hub

<img alt="hub-overview" width="1000" caption="The models page of the Hugging Face Hub, showing filters on the left and a list of models on the right." src="images/chapter01_hub-overview.png" id="hub-overview"/> 

<img alt="hub-model-card" width="1000" caption="A example model card from the Hugging Face Hub. The inference widget is shown on the right, where you can interact with the model." src="images/chapter01_hub-model-card.png" id="hub-model-card"/> 

### Hugging Face Tokenizers

### Hugging Face Datasets

### Hugging Face Accelerate

## Main Challenges with Transformers

## Conclusion

---